In [1]:
#
#todo 1 - CRIAR CONFIGS, UTILS E ETC
#todo 2 - Funções
#todo 3 - Pesquisar se possui variações no link do ml "/variations" e caso possua, capturar o nome da variação para incluir no tiny
#todo 4 - Ver no tiny, como incluo uma variação

In [2]:
from configs import *
from utils import *
import requests

### PROCESSAMENTO DE DADOS MERCADO LIVRE
#### 1. CAPTURA OS DADOS ATRAVÉS DO CÓDIGO MLB E COLA NA PLANILHA

In [3]:
spreadsheet_id = '1yAbwVoG3H1F70UTVSqMVLfmAjWEL0f7viqfqZfZAsys'

In [4]:
#! Autenticação Google Sheets

sheets_client = authenticate_gsheets(service_account_json , sheets_scopes)
sheets_client

A função authenticate_gsheets foi chamada!


In [5]:
#! Capturando dados do projeto
sheet_configs  = sheets_client.open_by_key(spreadsheet_id)
wks_busca_info  = sheet_configs.worksheet_by_title(title="Buscador de Informações")
wks_busca_info

<Worksheet 'Buscador de Informações' index:7>

In [6]:
lista_anuncios = get_active_products(wks_busca_info)
lista_anuncios

Executando a função get_active_products
Função get_active_products executada com sucesso!


['MLB3562637891', 'MLB4969200524']

In [7]:
search_result = get_ml_data(lista_anuncios)
search_result

[['MLB3562637891',
  'Cooktop A Gás 5 Bocas Inox Bivolt 75x52cm - Electrolux',
  '1759.9',
  'Electrolux',
  'Renove sua cozinha com o Cooktop a Gás Electrolux, uma fusão de elegância, desempenho e praticidade para transformar cada refeição em uma experiência única. Confira as características que tornam este cooktop a escolha perfeita para sua casa:\n\nCaracterísticas Principais:\n\nAlimentação: Elétrica, garantindo eficiência no preparo dos alimentos.\nLargura do Nicho para Cooktop: 69.5 cm, proporcionando encaixe perfeito em sua bancada.\nProfundidade do Nicho para Cooktop: 15 cm, ideal para otimizar o espaço na sua cozinha.\nMarca: Electrolux, sinônimo de qualidade e inovação.\nLargura e Profundidade: 75,00 cm x 52,00 cm, oferecendo ampla área de preparo.\nDimensão: 75x52 cm, tamanho ideal para diversas receitas.\nTensão Elétrica: Bivolt, adaptando-se às diferentes instalações elétricas.\nAcabamento: Inox, conferindo durabilidade e sofisticação.\nQuantidade de Bocas: 5 bocas, propor

In [8]:
paste_values(wks_busca_info, search_result)

In [9]:
df_produtos_final_step1 = get_products_df(wks_busca_info)
df_produtos_final_step1

,CÓDIGO Produto,Título,Preço,Marca,Descrição,Categoria ML,Gtin / EAN,Url Capa,Dimensões Url capa,Url2,...,Dimensões Url3,Url4,Dimensões Url4,Tem Variação,Variações,Dimensões,Altura,Largura,Comprimento,Peso
0,MLB3562637891,Cooktop A Gas 5 Bocas Inox Bivolt 75x52cm - El...,1759.9,Electrolux,Renove sua cozinha com o Cooktop a Gas Electro...,MLB120316,N/A,http://http2.mlstatic.com/D_630052-MLA40087737...,Ok - 500x259,http://http2.mlstatic.com/D_671687-MLU69967479...,...,Ok - 500x500,http://http2.mlstatic.com/D_953717-MLU72677733...,Ok - 500x500,Sim,Cor:Inox;Voltagem:127V/220V,80x50x20 - 9kg,80,50,20,9
1,MLB4969200524,Cooktop A Gas 4 Bocas Inox Bivolt 60x52cm - El...,1499.9,Electrolux,"""Desperte o Chef em Voce com o Cooktop a Gas 5...",MLB120316,N/A,http://http2.mlstatic.com/D_896648-MLB78219352...,Ok - 500x429,http://http2.mlstatic.com/D_964450-MLB77994718...,...,Ok - 500x364,http://http2.mlstatic.com/D_996924-MLB78219352...,Ok - 500x500,Sim,Cor:Prateado;Voltagem:110V/220V,60x50x20 - 7kg,60,50,20,7


In [10]:
import os
import requests
from PIL import Image
from io import BytesIO
import pandas as pd

# Função para converter números em letras
def number_to_letter(number):
    if number <= 9:
        return str(number)
    else:
        # Converte números maiores que 9 para letras
        return chr(65 + (number - 10))

# Supondo que o DataFrame já esteja disponível
df = df_produtos_final_step1

# Defina o caminho para a pasta de downloads
download_path = os.path.join(os.path.expanduser('~'), 'Downloads')

# Função para baixar e redimensionar a imagem
def download_and_resize_image(url, file_name):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            # Redimensiona a imagem para 500x500 usando o filtro LANCZOS
            image = image.resize((500, 500), Image.LANCZOS)
            image.save(file_name)
            print(f"Imagem salva e redimensionada: {file_name}")
        else:
            print(f"Erro ao baixar {url}: Status {response.status_code}")
    except Exception as e:
        print(f"Erro ao processar {url}: {e}")

# Itere sobre o DataFrame e faça o download e redimensionamento das imagens
for index, row in df.iterrows():
    base_name = number_to_letter(index + 1)  # Nome baseado no número da linha convertido em letra

    urls = [row['Url Capa'], row.get('Url2'), row.get('Url3'), row.get('Url4')]

    for i, url in enumerate(urls, start=1):
        if pd.notna(url):  # Verifica se a URL não está vazia
            # Nome do arquivo baseado no número da linha repetido conforme o índice da coluna
            file_name = os.path.join(download_path, base_name + number_to_letter(i) + '.jpg')
            download_and_resize_image(url, file_name)


Imagem salva e redimensionada: C:\Users\Usuário\Downloads\11.jpg
Imagem salva e redimensionada: C:\Users\Usuário\Downloads\12.jpg
Imagem salva e redimensionada: C:\Users\Usuário\Downloads\13.jpg
Imagem salva e redimensionada: C:\Users\Usuário\Downloads\14.jpg
Imagem salva e redimensionada: C:\Users\Usuário\Downloads\21.jpg
Imagem salva e redimensionada: C:\Users\Usuário\Downloads\22.jpg
Imagem salva e redimensionada: C:\Users\Usuário\Downloads\23.jpg
Imagem salva e redimensionada: C:\Users\Usuário\Downloads\24.jpg


In [11]:
categorias = get_df_categories()

In [12]:
df_categorias = mapeamento_categoria(df_produtos_final_step1,categorias)

### INTEGRANDO TINY COM OS DADOS DO ML NA PLANILHA
#### 1. CAPTURA OS DADOS DA PLANILHA, SUBSTITUI PELOS DADOS NECESSÁRIO NO TINY E CRIA O JSON


In [13]:
# Cria o JSON com base nos dados do DataFrame
produto_info = criar_json_produtos(df_categorias)
produto_info
# Visualiza o JSON gerado
#print(json.dumps(produtos_info, indent=4))

{'produtos': [{'produto': {'sequencia': 1,
    'nome': 'Cooktop A Gas 5 Bocas Inox Bivolt 75x52cm - Electrolux',
    'codigo': 'MLB3562637891',
    'gtin': 'N/A',
    'unidade': 'UN',
    'preco': 1759.9,
    'peso_liquido': 9.0,
    'peso_bruto': 9.0,
    'altura_embalagem': 80.0,
    'largura_embalagem': 50.0,
    'comprimento_embalagem': 20.0,
    'origem': '0',
    'situacao': 'A',
    'tipo': 'P',
    'estoque_atual': 100,
    'estoque_minimo': 100,
    'dias_preparacao': 3,
    'categoria': 'Eletrodomésticos >> Fornos e Fogões >> Cooktop',
    'descricao_complementar': 'Renove sua cozinha com o Cooktop a Gas Electrolux, uma fusao de elegancia, desempenho e praticidade para transformar cada refeicao em uma experiencia unica. Confira as caracteristicas que tornam este cooktop a escolha perfeita para sua casa:\n\nCaracteristicas Principais:\n\nAlimentacao: Eletrica, garantindo eficiencia no preparo dos alimentos.\nLargura do Nicho para Cooktop: 69.5 cm, proporcionando encaixe perfei

In [14]:
#
#! Conectando ao tiny e criando os produtos
connect_and_create_tiny(produto_info)

Erro na solicitação. Código de erro: 2
[{'registro': {'sequencia': '1', 'status': 'Erro', 'codigo_erro': '30', 'erros': [{'erro': 'Registro em duplicidade - código (SKU) do produto já cadastrado'}]}}, {'registro': {'variacoes': [{'variacao': {'id': 888931845}}], 'sequencia': '2', 'status': 'OK', 'id': 888931844}}]


## **REANUNCIOS**

In [17]:

#! Capturando dados do projeto
sheet_configs  = sheets_client.open_by_key(spreadsheet_id)
wks_reanuncio_info  = sheet_configs.worksheet_by_title(title="REANUNCIOS - Buscador de Informações")

In [18]:
reanuncios_pendentes = reanuncio_get_active_products(wks_reanuncio_info)
reanuncios_pendentes

Executando a função get_active_products


['MLB4145506666', 'MLB2771605778']

In [19]:
reanuncio_search_result = renuncio_get_ml_data(reanuncios_pendentes)

In [20]:
reanuncio_paste_values(wks_reanuncio_info, reanuncio_search_result)

In [21]:
#! Consulta a planilha de buscador de informações com as infos preenchidas
#! Faz o tratamento para separar altura, largura, comprimento e peso e cria um dataframe
def reanuncio_get_products_df(wks_reanuncio_info):
    products_df                                                 = wks_reanuncio_info.get_as_df(has_header=True, start="B5", end="Q50")

    final_products_df = products_df

    return(final_products_df)
#end def

In [22]:
reanuncio_df = reanuncio_get_products_df(wks_reanuncio_info)
reanuncio_df

,MLB Antigo,Título,Descrição,Marca,Preço,Imagem1,Imagem2,Imagem3,Imagem4,Imagem5,Imagem6,Imagem7,Imagem8,Imagem9,Imagem10,Imagem11
0,MLB4145506666,Cooler Caixa Térmica Alça Suporte 28 L 44x44x3...,O que vem a seguir na sua lista de aventuras? ...,Igloo,699.9,http://http2.mlstatic.com/D_646559-MLA77553898...,http://http2.mlstatic.com/D_980105-MLB77553827...,http://http2.mlstatic.com/D_781338-MLB77771995...,http://http2.mlstatic.com/D_888387-MLB77772006...,http://http2.mlstatic.com/D_637137-MLB77772065...,,,,,,
1,MLB2771605778,Cooler Caixa Térmica 86 Litros Rodas Aço E Sup...,Esta caixa térmica resistente da Igloo é ótima...,Igloo,1789.9,http://http2.mlstatic.com/D_799353-MLB73333293...,http://http2.mlstatic.com/D_837123-MLB73333596...,http://http2.mlstatic.com/D_840623-MLB73247065...,http://http2.mlstatic.com/D_633672-MLB73333596...,http://http2.mlstatic.com/D_749366-MLB73333263...,http://http2.mlstatic.com/D_929111-MLB73247115...,,,,,


In [23]:
import os
import requests
from PIL import Image
from io import BytesIO
import pandas as pd

# Função para converter números em letras
def number_to_letter(number):
    if number <= 9:
        return str(number)
    else:
        # Converte números maiores que 9 para letras
        return chr(65 + (number - 10))

# Supondo que o DataFrame já esteja disponível
df = reanuncio_df

# Defina o caminho para a pasta de downloads
download_path = os.path.join(os.path.expanduser('~'), 'Downloads')

# Função para baixar e redimensionar a imagem
def download_and_resize_image(url, file_name):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            # Redimensiona a imagem para 500x500 usando o filtro LANCZOS
            image = image.resize((500, 500), Image.LANCZOS)
            image.save(file_name)
            print(f"Imagem salva e redimensionada: {file_name}")
        else:
            print(f"Erro ao baixar {url}: Status {response.status_code}")
    except Exception as e:
        print(f"Erro ao processar {url}: {e}")

# Função principal para iterar sobre o DataFrame e baixar as imagens das colunas Imagem1 até Imagem11
def baixar_imagens(df):
    # Itera sobre o DataFrame e processa as colunas de imagens
    for index, row in df.iterrows():
        base_name = number_to_letter(index + 1)  # Nome baseado no número da linha convertido em letra

        # Captura as colunas de Imagem1 a Imagem11
        urls = [row.get(f'Imagem{i}') for i in range(1, 12)]  # Imagem1 a Imagem11

        # Itera sobre as URLs de imagens e faz o download e redimensionamento
        for i, url in enumerate(urls, start=1):
            if pd.notna(url):  # Verifica se a URL não está vazia
                # Nome do arquivo baseado no número da linha e na coluna da imagem
                file_name = os.path.join(download_path, base_name + number_to_letter(i) + '.jpg')
                download_and_resize_image(url, file_name)

# Chamada da função para baixar as imagens
baixar_imagens(df)

Imagem salva e redimensionada: C:\Users\Usuário\Downloads\11.jpg
Imagem salva e redimensionada: C:\Users\Usuário\Downloads\12.jpg
Imagem salva e redimensionada: C:\Users\Usuário\Downloads\13.jpg
Imagem salva e redimensionada: C:\Users\Usuário\Downloads\14.jpg
Imagem salva e redimensionada: C:\Users\Usuário\Downloads\15.jpg
Erro ao processar : Invalid URL '': No scheme supplied. Perhaps you meant https://?
Erro ao processar : Invalid URL '': No scheme supplied. Perhaps you meant https://?
Erro ao processar : Invalid URL '': No scheme supplied. Perhaps you meant https://?
Erro ao processar : Invalid URL '': No scheme supplied. Perhaps you meant https://?
Erro ao processar : Invalid URL '': No scheme supplied. Perhaps you meant https://?
Erro ao processar : Invalid URL '': No scheme supplied. Perhaps you meant https://?
Imagem salva e redimensionada: C:\Users\Usuário\Downloads\21.jpg
Imagem salva e redimensionada: C:\Users\Usuário\Downloads\22.jpg
Imagem salva e redimensionada: C:\Users\U